In [1]:
import pandas as pd
import numpy as np
%matplotlib inline 
import spacy
import re
from functools import reduce


In [40]:
# Data obtained from Figure Eight.  Contains economic news articles that were categorized as either having information on the US economy or not.
data = pd.read_csv("Economic-News.csv",encoding = "ISO-8859-1")

#link: https://www.figure-eight.com/wp-content/uploads/2016/03/Full-Economic-News-DFE-839861.csv

In [3]:
data

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
0,842613455,False,finalized,3,12/5/15 17:48,3.0,0.6400,yes,0.6400,wsj_398217788,8/14/91,Yields on CDs Fell in the Latest Week,NaN,NaN,NEW YORK -- Yields on most certificates of dep...
1,842613456,False,finalized,3,12/5/15 16:54,NaN,NaN,no,1.0000,wsj_399019502,8/21/07,The Morning Brief: White House Seeks to Limit ...,NaN,NaN,The Wall Street Journal Online</br></br>The Mo...
2,842613457,False,finalized,3,12/5/15 1:59,NaN,NaN,no,1.0000,wsj_398284048,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,NaN,NaN,WASHINGTON -- In an effort to achieve banking ...
3,842613458,False,finalized,3,12/5/15 2:19,NaN,0.0000,no,0.6750,wsj_397959018,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,NaN,NaN,The statistics on the enormous costs of employ...
4,842613459,False,finalized,3,12/5/15 17:48,3.0,0.3257,yes,0.6400,wsj_398838054,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NaN,NaN,NEW YORK -- Indecision marked the dollar's ton...
5,842613460,False,finalized,3,12/4/15 23:15,3.0,0.6783,yes,1.0000,wsj_905654974,11/23/11,"Stocks Fall Again; BofA, Alcoa Slide",NaN,NaN,"Stocks declined, as investors weighed slower-t..."
6,842613461,False,finalized,3,12/5/15 7:35,NaN,0.0000,no,0.6496,wsj_398739166,5/24/01,"Bank of Montreal, Royal Bank Profits Rose in 2...",NaN,NaN,TORONTO -- Royal Bank of Canada and Bank of Mo...
7,842613462,False,finalized,3,12/5/15 17:22,NaN,NaN,no,1.0000,wsj_397957465,10/22/86,Battle Over Medical Costs Isn't Over,NaN,NaN,Many people think that the monster of health-c...
8,842613463,False,finalized,3,12/4/15 23:07,NaN,NaN,no,1.0000,wsj_398737834,5/30/01,Sequenom to Buy Gemini Genomics In Stock Accord,NaN,NaN,"Sequenom Inc., a genomics-based biotechnology ..."
9,842613464,False,finalized,3,12/5/15 18:40,4.0,0.6657,yes,1.0000,wsj_397912506,11/1/84,U.S. Dollar Falls Against Most Currencies; Dec...,NaN,NaN,The U.S. dollar declined against most major fo...


In [4]:
data.shape

(8000, 15)

In [5]:
# Check for imbalance
data["relevance"].value_counts()


no          6571
yes         1420
not sure       9
Name: relevance, dtype: int64

In [6]:
text= data["text"]

In [7]:
# See text to determine cleaning needed.
text[0]

'NEW YORK -- Yields on most certificates of deposit offered by major banks dropped more than a tenth of a percentage point in the latest week, reflecting the overall decline in short-term interest rates.</br></br>On small-denomination, or "consumer," CDs sold directly by banks, the average yield on six-month deposits fell to 5.49% from 5.62% in the week ended yesterday, according to an 18-bank survey by Banxquote Money Markets, a Wilmington, Del., information service.</br></br>On three-month "consumer" deposits, the average yield sank to 5.29% from 5.42% the week before, according to Banxquote. Two banks in the Banxquote survey, Citibank in New York and CoreStates in Pennsylvania, are paying less than 5% on threemonth small-denomination CDs.</br></br>Declines were somewhat smaller on five-year consumer CDs, which eased to 7.37% from 7.45%, Banxquote said.</br></br>Yields on three-month and six-month Treasury bills sold at Monday\'s auction plummeted more than a fifth of a percentage po

In [8]:
# See text to determine cleaning needed.
text[100]

'J.P. Morgan Chase & Co. reached a tentative deal this weekend to pay $13 billion to end a number of civil investigations into its sale of mortgage securities before the 2008 financial crisis, but a separate and potentially more serious criminal probe into the bank and its executives will continue.</br></br>The Justice Department, convinced it has strong evidence related to the bank\'s conduct and eager to send a message to Wall Street, rebuffed repeated attempts by J.P. Morgan to settle the criminal investigation without admitting wrongdoing and agreed only to resolve the civil investigations. It also threatened last Thursday to file its civil case this coming Wednesday if the two sides can\'t reach a final deal, said people close to the talks.</br></br>The proposed pact includes $4 billion to settle claims by the Federal Housing Finance Agency that J.P. Morgan misled Fannie Mae and Freddie Mac about the quality of the mortgage securities it sold them, another $4 billion in consumer r

In [9]:
# See text to determine cleaning needed.
text[3000]

"South Korea's efforts to engineer an economic recovery received a shot in the arm yesterday when global investors turned out in droves to participate in the country's landmark $4 billion bond sale.</br></br>In addition to paving the way for more South Korean corporate bonds, the sale is expected to set the stage for billions of dollars in Asian bonds in the months ahead.</br></br>Searching for high-yielding bonds in an age of near-record-low bond yields, investors from around the globe showed strong demand for yesterday's two-part deal. Seoul sold $1 billion of five-year notes at a yield of 8.953%, or 3.45 percentage points above comparable U.S. Treasurys. Some $3 billion of 10-year securities were sold at a yield of 9.084%, or 3.55 percentage points above Treasurys.</br></br>Seoul was able to sell the bonds at a yield just below the level expected several days ago, underscoring strong demand for the bonds despite last week's downgrade of Japan's debt outlook by Moody's Investors Serv

In [10]:
# See text to determine cleaning needed.
text[4000]

'Ted Nigh wants his friends to know that life in the nursing home is all right and that they shouldn\'t hesitate to visit him. He is getting good care and gets along well with the staff.</br></br>Nigh is 20 years old, a 2002 graduate of Falls Church High School. He is also a quadriplegic with cerebral palsy. With no programs available to help Nigh after graduation, his father, a single parent, felt he had little choice but to put his son in a nursing home that accepts Medicaid patients. Sam Nigh, 54, who was recently laid off from his job at an insurance company, said he hopes that some alternative will come through next year.</br></br>"They\'re doing a good job taking care of me," Ted Nigh said of the 180-bed nursing home in Manassas where he has lived since May. But his stay "is becoming too long," he added. "You just get aggravated. . . . I don\'t want nursing home life to become a regular habit."</br></br>Like all states, Virginia is required by federal law to provide special educa

In [11]:
# See text to determine cleaning needed.
text[5000]

'Financially troubled low-cost carrier Independence Air painted a gloomy picture of its future yesterday, saying it may be forced to file for bankruptcy protection.</br></br>The airline\'s Dulles-based parent, Flyi Inc., said in its quarterly filing with the Securities and Exchange Commission that it won\'t be able to pay its bills unless it soon raises "significant funds." "Such matters raise substantial doubt about the Company\'s ability to continue as a going concern," the filing said. It added that if the situation continues, it will have to file for bankruptcy protection.</br></br>Some Wall Street analysts have been predicting since last fall that Flyi may be forced to file for bankruptcy protection. The airline has raised the possibility, too, but never with the sense of urgency it demonstrated yesterday.</br></br>In a mid-afternoon conference call with Wall Street analysts before the quarterly report was filed, Flyi chief executive Kerry B. Skeen didn\'t mention bankruptcy. But 

In [14]:
# Extra article text from dataframe
text= data["text"]

In [38]:
# Clean the text
def text_cleaner(text):
    text = text.apply(lambda x:re.sub("</br></br>", " ", x))
    text = text.apply(lambda x:re.sub("--", "", x))
    text = text.apply(lambda x:re.sub("Author:", " ", x))
    text = text.apply(lambda x:re.sub("By:", " ", x))
    return text
cleaned_text=text_cleaner(text)
    

In [39]:
# See cleaned text
cleaned_text

0       NEW YORK  Yields on most certificates of depos...
1       The Wall Street Journal Online The Morning Bri...
2       WASHINGTON  In an effort to achieve banking re...
3       The statistics on the enormous costs of employ...
4       NEW YORK  Indecision marked the dollar's tone,...
5       Stocks declined, as investors weighed slower-t...
6       TORONTO  Royal Bank of Canada and Bank of Mont...
7       Many people think that the monster of health-c...
8       Sequenom Inc., a genomics-based biotechnology ...
9       The U.S. dollar declined against most major fo...
10      The Wall Street Journal Online A LATE POP by G...
11      MCI WorldCom Inc., fueled by growth in Interne...
12        James B. Stewart The dreaded "D" word is bac...
13      NEW YORK  Commercial and industrial loan volum...
14      WASHINGTON  A week before an international eco...
15      According to Kevin A. Hassett and Aparna Mathu...
16      OTAVALO, Ecuador  The long black braid, the bl...
17      NEW YO

In [20]:
#Combined rows into one string for NLP parsing.
combined_text = reduce(lambda x, y: x+y, cleaned_text)

In [21]:
nlp_df = pd.DataFrame()
nlp_df["cleaned_text"] = cleaned_text

In [23]:
#Load English Spacy
nlp = spacy.load("en")

In [36]:
#Increase max length(receive error otherwise), the process
nlp.max_length = 11000000
nlp_text = nlp(combined_text)

MemoryError: 